## Camilla's Exploratory Graphs and XGBoost Model

In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

mergdf = pd.read_csv("merged3.csv")

In [5]:
mergdf.head(2)

,date,count,name,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,description,icon,stations,full_moon,Holiday,Day of Week,CHIC917URN,day_of_week,year,month
0,2010-01-01,13.0,"Chicago,United States",-8.3,-14.5,-11.7,-15.0,-22.4,-19.0,-16.1,...,Partly cloudy throughout the day.,partly-cloudy-day,"72534014819,KORD,KMDW,72530094846,74466504838,...",NaN,New Year's Day,Friday,12.2,4,2010,1
1,2010-01-02,4.0,"Chicago,United States",-11.4,-16.4,-14.1,-18.4,-25.3,-22.6,-19.6,...,Partly cloudy throughout the day.,partly-cloudy-day,"72534014819,KORD,KMDW,72530094846,74466504838,...",NaN,NaN,NaN,12.2,5,2010,1


In [7]:
X = mergdf[['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'humidity',
             'precip', 'precipprob', 'precipcover', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 
             'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 
             'moonphase', 'day_of_week', 'year', 'month', 'preciptype', 'sunrise', 'sunset', 'conditions', 'full_moon', 
             'Holiday', 'CHIC917URN']]

num_features = mergdf[['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'humidity',
                        'precip', 'precipprob', 'precipcover', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 
                        'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 
                        'moonphase', 'day_of_week', 'year', 'month', 'CHIC917URN']]

cat_features = mergdf[['preciptype', 'conditions', 'full_moon', 'Holiday']]

cat_features = X.select_dtypes(include="object").columns
num_features = X.select_dtypes(exclude="object").columns

y = list(mergdf['count'])

In [8]:
# Assuming you have your data loaded into X (features) and y (target variable)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into DMatrix format, which is required for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define the parameters for XGBoost
params = {
    'objective': 'reg:squarederror', # Regression task
    'eval_metric': 'rmse' # Root Mean Squared Error as evaluation metric
}

# Train the XGBoost model
num_round = 100 # Number of boosting rounds
bst = xgb.train(params, dtrain, num_round)

# Make predictions on the test set
predictions = bst.predict(dtest)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:preciptype: object, sunrise: object, sunset: object, conditions: object, full_moon: object, Holiday: object